In [1]:
import time
class Timer:
    def __init__(self):
        return
    def __enter__(self):
        self._t0 = time.perf_counter()
        return
    def __exit__(self, type, value, traceback):
        t1 = time.perf_counter()
        print(f"Elapsed: {t1-self._t0:0.3f} seconds.")
        return
    pass

In [2]:
from fmu.sumo.explorer import Explorer
exp=Explorer(env="preview")

/Users/RAYW/py-envs/explorer/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [3]:
def total_blob_size(sc):
    tbs = sc.metrics.sum("file.size_bytes")
    if tbs == 0:
        tbs = sc.metrics.sum("_sumo.blob_size")
    return tbs

def do_aggregate(tagname, rels, columns):
    print(f"{tagname}: {len(rels)} objects, {len(rels.columns)} columns.")
    tot_size_bytes = total_blob_size(rels)
    print(f"Total size of input: {tot_size_bytes / (1024*1024*1024):.3f} GiB")
    with Timer():
        agg=rels.filter(column=columns).aggregate(columns=columns)
    print(agg.to_pandas().sort_values(by=["REAL", "DATE"]))

def run_exp(caseuuid, itername, tagname, columns):
    case = exp.get_case_by_uuid(caseuuid)
    print(f"{case.asset}: {case.name}: {caseuuid}")
    rels=case.tables.filter(iteration=itername, realization=True, tagname=tagname, 
                            complex={"bool": {"must_not": [{"term": {"_sumo.hidden": True}}]}})
    do_aggregate(tagname, rels, columns)
    rels=case.tables.filter(iteration=itername, realization=True, tagname=tagname,
                            complex={"term": {"_sumo.hidden": True}})
    do_aggregate(tagname, rels, columns)

In [4]:
run_exp("359e7c72-a4ca-43ee-9203-f09cd0f149a9", "pred-0", "summary", ["FOPT"])

Troll: 24.0.0-20240828_ix_network_test5: 359e7c72-a4ca-43ee-9203-f09cd0f149a9
summary: 27 objects, 64996 columns.
Total size of input: 1.248 GiB
Elapsed: 15.166 seconds.
          DATE         FOPT  REAL
137 2024-07-02  282442208.0     6
138 2024-07-03  282451072.0     6
139 2024-08-01  282677120.0     6
140 2024-09-01  282889760.0     6
141 2024-10-01  283077440.0     6
..         ...          ...   ...
47  2025-02-15  286229120.0   249
48  2025-04-01  286425696.0   249
49  2025-09-01  287060416.0   249
50  2025-10-01  287176832.0   249
51  2026-01-01  287523552.0   249

[265 rows x 3 columns]
summary: 3537 objects, 64996 columns.
Total size of input: 1.087 GiB
Elapsed: 1.692 seconds.
          DATE         FOPT  REAL
52  2024-07-02  282442208.0     6
53  2024-07-03  282451072.0     6
54  2024-08-01  282677120.0     6
55  2024-09-01  282889760.0     6
56  2024-10-01  283077440.0     6
..         ...          ...   ...
173 2025-02-15  286229120.0   249
174 2025-04-01  286425696.0   249

In [5]:
run_exp("fc6cc7d3-6162-46a3-9d69-48ad1eaecdfb", "iter-0", "summary", ["FOPT"])

Troll: 24.0.0-20240820: fc6cc7d3-6162-46a3-9d69-48ad1eaecdfb
summary: 196 objects, 24568 columns.
Total size of input: 30.013 GiB
Elapsed: 32.124 seconds.
             DATE          FOPT  REAL
708796 1990-02-01  0.000000e+00     1
708797 1990-03-01  1.445590e+05     1
708798 1990-04-01  2.741935e+05     1
708799 1990-05-01  4.145006e+05     1
708800 1990-06-01  5.512956e+05     1
...           ...           ...   ...
841571 2024-06-27  2.980280e+08   249
841572 2024-06-28  2.980311e+08   249
841573 2024-06-29  2.980342e+08   249
841574 2024-06-30  2.980384e+08   249
841575 2024-07-01  2.980405e+08   249

[952560 rows x 3 columns]
summary: 9800 objects, 24568 columns.
Total size of input: 29.907 GiB
Elapsed: 4.722 seconds.
             DATE          FOPT  REAL
34020  1990-02-01  0.000000e+00     1
34021  1990-03-01  1.445590e+05     1
34022  1990-04-01  2.741935e+05     1
34023  1990-05-01  4.145006e+05     1
34024  1990-06-01  5.512956e+05     1
...           ...           ...   ...
31

In [6]:
run_exp("5b558daf-61c5-400a-9aa2-c602bb471a16", "iter-0", "summary", ["FOPT"])

Drogon: ruaj_testcase: 5b558daf-61c5-400a-9aa2-c602bb471a16
summary: 160 objects, 974 columns.
Total size of input: 0.175 GiB
Elapsed: 2.485 seconds.
            DATE          FOPT  REAL
4910  2018-01-01  0.000000e+00     0
4911  2018-01-02  0.000000e+00     0
4912  2018-01-05  0.000000e+00     0
4913  2018-01-06  3.991868e+03     0
4914  2018-01-09  1.596676e+04     0
...          ...           ...   ...
36831 2020-06-14  7.278816e+06   159
36832 2020-06-27  7.349246e+06   159
36833 2020-06-28  7.354664e+06   159
36834 2020-06-30  7.365482e+06   159
36835 2020-07-01  7.370888e+06   159

[39280 rows x 3 columns]
summary: 320 objects, 974 columns.
Total size of input: 0.163 GiB
Elapsed: 2.528 seconds.
            DATE          FOPT  REAL
19394 2018-01-01  0.000000e+00     0
19395 2018-01-02  0.000000e+00     0
19396 2018-01-05  0.000000e+00     0
19397 2018-01-06  3.991868e+03     0
19398 2018-01-09  1.596676e+04     0
...          ...           ...   ...
10795 2020-06-14  7.278816e+06 